<a rel="license" href="http://creativecommons.org/licenses/by-sa/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-sa/4.0/80x15.png" /></a><div align="center">This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-sa/4.0/">Creative Commons Attribution-ShareAlike 4.0 International License</a>.</div>

----

The ESC403 cluster comes with [Catherine Devlin's `%sql`-magic for IPython][1]; this allows you to run SQL queries from the IPython notebook, and intermix them with Python code.

[1]: https://github.com/catherinedevlin/ipython-sql

Before we can use the `%sql` syntax, two steps must be taken:

* Load the IPython-SQL bridge code

In [1]:
%load_ext sql


* Connect to an actual database; this must be the first `%sql` statement (the funny `rmurri@/lustre` syntax is correct and means "connect to the PostGreSQL DB named `lustre` running on *this* host as user `rmurri`"  -- please replace `rmurri` with your local user name):

In [2]:
%sql postgresql://rmurri@/lustre


u'Connected: rmurri@lustre'

Now you can run 1-line SQL queries by prefixing them with `%sql`:

In [3]:
%sql select * from lustre_sample limit 5;

5 rows affected.


usr,grp,atime,mtime,blksize,size,path
usr296,i5105,1390288156,1390288310,1212,1240449,/scratch/bioc/usr296/actin/protein_reid-unbound_5/distances/wordom_dist_reidi_res_10.xvg
usr296,i5105,1390290448,1390290602,1212,1240449,/scratch/bioc/usr296/actin/protein_reid-unbound_5/distances/wordom_dist_reidi_ring_res_121.xvg
usr296,i5105,1390292628,1390292786,1212,1240449,/scratch/bioc/usr296/actin/protein_reid-unbound_5/distances/wordom_dist_reidi_ring_res_233.xvg
usr296,i5105,1390288351,1390288509,1212,1240449,/scratch/bioc/usr296/actin/protein_reid-unbound_5/distances/wordom_dist_reidi_tail_res_13.xvg
usr296,i5105,1390292817,1390292971,1212,1240449,/scratch/bioc/usr296/actin/protein_reid-unbound_5/distances/wordom_dist_reidi_res_252.xvg


It is also possible to run multi-line (or multiple) SQL queries by using the `%%sql` syntax instead.  Note that in this case the SQL instructions *must not* be on the same line as the `%%sql` magic marker:

In [ ]:
%%sql
select count(*) from lustre;
select distinct count(usr) from lustre;
select distinct count(grp) from lustre;

The `%%time` magic prints the time taken to evaluate a cell (which comes handy when doing performance comparisons):

In [ ]:
%%time

import time
time.sleep(5)

----

**Note:** to keep running times low, we will be using table `lustre_sample` throughout, which contains a sample of 5% the rows of the original `lustre` table.

## 1. Is it possible to convert fields atime and mtime to PostgreSQL's TIMESTAMP type?

Yes, it *is* possible to alter a SQL table definition after the table has been created.  Look at the documentation for the [ALTER TABLE](https://www.tutorialspoint.com/sql/sql-alter-command.htm) statement.

We shall break this down into steps: (1) create a new table, (2) populate it, then (3) alter the definition and (4) fill the new column with values.

In [5]:
# (1) create a new table and populate it
%sql create temporary table files(path VARCHAR(255), atime INT)

Done.


[]

In [6]:
# (2) populate it with a copy of the data
%sql insert into files(path, atime) select path, atime from lustre_sample

1519053 rows affected.


[]

In [9]:
# (3) alter table definition: add new columns for "access time" using the TIMESTAMP type
%sql alter table files add access_time TIMESTAMP

Done.


[]

In [11]:
# (4) populate additional column
%sql update files set access_time = to_timestamp(atime)

1519053 rows affected.


[]

Show some data from the table we created::

In [15]:
%sql select * from files limit 3

3 rows affected.


path,atime,access_time
/scratch/bioc/usr25/intel/ism/bin/installwrapper.sh,1367575781,2013-05-03 10:09:41
/scratch/pci/usr394/cp2k/cp2k/src/sort_m.f90,1392886784,2014-02-20 08:59:44
/scratch/pci/usr360/test/BASIS_MOLOPT,1351174511,2012-10-25 14:15:11


## 2. Can you count the number of files in a given directory?

Yes, using SQL's `like` string matching operator, which allows any
part of a string to be matched by the `%` character (i.e., just like
`*` for file names)::

In [14]:
%sql SELECT count(path) FROM lustre_sample WHERE path LIKE '/scratch/econ/H2/%'

1 rows affected.


count
132


## 3. Can you find the directory that holds the largest number of files? 

Let's just sketch a general plan:
- Split the `path` column into separate columns *directory name* and *file name*,
- then group by *directory name*
and count.

This is a simple solution, however:

* It relies on the availability of functions to split directory from file name, which may not be available in all SQL engines:

        %sql SELECT DIRNAME(path) AS dirpath, COUNT(FILENAME(path)) FROM lustre GROUP BY dirpath

But more importantly:

* Only allows counting the number of immediate "children" of a directory.  If one wishes to count the number of files in a given directory *tree*, then this is note easy to do with SQL. See <http://en.wikipedia.org/wiki/Nested_set_model> for more information.  It is, however, a straightforward exercise with Map/Reduce.

## 4. Can you find the directory tree that holds the largest number of files?

No. We would need to find another representation for the path names, that allows recovering directory components up to any depth.  The split dirname/basename suggested above only allows counting the number of immediate "children" of a directory.  If one wishes to count the number of files in a given directory *tree*, then this is note easy to do with SQL. See <http://en.wikipedia.org/wiki/Nested_set_model> for more information.  

This is, however, a straightforward exercise with Map/Reduce.